A Regressão Logística é um algoritmo de classificação de Aprendizado de Máquina usado para prever a probabilidade de uma variável dependente categórica. Na regressão logística, a variável dependente é uma variável binária que contém dados codificados como 1 (sim, sucesso, etc.) ou 0 (não, falha, etc.). Em outras palavras, o modelo de regressão logística prevê P(Y=1) como uma função de X.


## Pressupostos da Regressão Logística:

* Regressão Logística Binária requer que a variável dependente seja binária.
* Para uma regressão binária, o nível do fator 1 da variável dependente deve representar o resultado desejado.
* Apenas as variáveis relevantes devem ser incluídas.
* As variáveis independentes devem ser independentes entre si. Ou seja, o modelo deve ter pouca ou nenhuma multicolinearidade.
* As variáveis independentes estão linearmente relacionadas com os log-odds. (Os log odds representam o logaritmo natural da razão entre a probabilidade de um evento ocorrer e a probabilidade de não ocorrer. Os log odds são frequentemente usados na regressão logística porque permitem transformar a relação não-linear entre as probabilidades e as variáveis independentes em uma relação linear).
* A regressão logística requer tamanhos de amostra bastante grandes.

In [1]:
# pandas para trabalhar com dataframes.
import pandas as pd

# numpy para operações numéricas eficientes e computação matricial.
import numpy as np

# sklearn.preprocessing para pré-processar os dados antes de alimentá-los em um modelo de aprendizado de máquina.
from sklearn import preprocessing

# matplotlib.pyplot para criar visualizações gráficas interativas.
import matplotlib.pyplot as plt 
plt.rc("font", size=14)  # Definindo o tamanho padrão da fonte dos gráficos.

# Importando a classe LogisticRegression do scikit-learn para realizar uma tarefa de classificação com regressão logística.
from sklearn.linear_model import LogisticRegression

# Importando a função train_test_split do scikit-learn para dividir os dados em conjuntos de treinamento e teste.
from sklearn.model_selection import train_test_split

# seaborn para criar gráficos estatísticos visuais.
import seaborn as sns
sns.set(style="white")  # Definindo o estilo de fundo dos gráficos seaborn como branco.
sns.set(style="whitegrid", color_codes=True)  # Definindo o estilo de grade branca para os gráficos seaborn.


In [2]:
df = pd.read_csv(r'https://raw.githubusercontent.com/MAI0881/datasets/main/telecom_churn.csv', encoding = 'utf-8', sep = ',')
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


## Cleansing data

In [3]:
df.replace(' ', pd.NA, inplace=True)

In [4]:
nan_counts = df.isnull().sum()
rows_with_nan = df[df.isnull().any(axis=1)]
#rows_with_nan

In [5]:
df.dropna(inplace=True)

In [6]:
df = df.drop(['customerID'], axis = 1)

In [7]:
df.nunique()

gender                 2
SeniorCitizen          2
Partner                2
Dependents             2
tenure                72
PhoneService           2
MultipleLines          3
InternetService        3
OnlineSecurity         3
OnlineBackup           3
DeviceProtection       3
TechSupport            3
StreamingTV            3
StreamingMovies        3
Contract               3
PaperlessBilling       2
PaymentMethod          4
MonthlyCharges      1584
TotalCharges        6530
Churn                  2
dtype: int64

In [8]:
df['Churn'] = df['Churn'].replace({'Yes': 1, 'No': 0}) # mudando para variável binária numérica para usar como variável dependente

In [9]:
df['SeniorCitizen'] = df['SeniorCitizen'].replace({1: 'Yes', 0: 'No'}) #mudando para variável binária categórica para usar como variável independente

In [ ]:
df

## Data exploration 

In [ ]:
import seaborn as sns

sns.countplot(x='InternetService', hue='Churn', data=df)
plt.show()

In [ ]:
df['Churn'].value_counts()

In [ ]:
sns.countplot(x='Churn',data=df, palette='hls')
plt.show()
plt.savefig('count_plot')

In [ ]:
count_no_churn = len(df[df['Churn']==0])
count_churn = len(df[df['Churn']==1])
pct_of_no_churn = count_no_churn/(count_no_churn+count_churn)
print("percentage of no churn is", pct_of_no_churn*100)
pct_of_churn = count_churn/(count_no_churn+count_churn)
print("percentage of churn", pct_of_churn*100)

In [ ]:
df.groupby('Dependents')[['tenure','MonthlyCharges','TotalCharges']].mean()

In [ ]:
df['TotalCharges'] = df['TotalCharges'].replace(' ', np.nan)

df['TotalCharges'] = df['TotalCharges'].astype(float)

In [ ]:
df.groupby('MultipleLines')[['tenure','MonthlyCharges','TotalCharges']].mean()

In [ ]:
df.groupby('InternetService')[['tenure','MonthlyCharges','TotalCharges']].mean()

In [ ]:
df.groupby('TechSupport')[['tenure','MonthlyCharges','TotalCharges']].mean()

In [ ]:
df.groupby('PaymentMethod')[['tenure','MonthlyCharges','TotalCharges']].mean()

In [ ]:
table=pd.crosstab(df.Dependents,df.Churn)
table.div(table.sum(1).astype(float), axis=0).plot(kind='bar', stacked=True)
plt.title('Churn Frequency for Dependents')
plt.xlabel('Dependents')
plt.ylabel('Frequency of Churn')
plt.savefig('churn_frequency_per_Dependents')

In [ ]:
table=pd.crosstab(df.Partner,df.Churn)
table.div(table.sum(1).astype(float), axis=0).plot(kind='bar', stacked=True)
plt.title('Churn Frequency for MultipleLines')
plt.xlabel('MultipleLines')
plt.ylabel('Frequency of Churn')
plt.savefig('churn_frequency_per_MultipleLines')

The frequency of churn depends a great deal on the internet service. Thus, the internet service can be a good predictor of the outcome variable.

In [ ]:
pd.crosstab(df.PaymentMethod,df.Churn).plot(kind='bar')
plt.title('Churn Frequency for PaymentMethod')
plt.xlabel('PaymentMethod')
plt.ylabel('Frequency of Churn')
plt.savefig('churn_frequency_per_PaymentMethod')

In [ ]:
table=pd.crosstab(df.TechSupport,df.Churn)
table.div(table.sum(1).astype(float), axis=0).plot(kind='bar', stacked=True)
plt.title('Churn Frequency for TechSupport')
plt.xlabel('TechSupport')
plt.ylabel('Frequency of Churn')
plt.savefig('churn_frequency_per_TechSupport')

In [ ]:
df.dtypes

In [ ]:
df.MonthlyCharges.hist()
plt.title('Histogram of MonthlyCharges')
plt.xlabel('MonthlyCharges')
plt.ylabel('Frequency')
plt.savefig('hist_MonthlyCharges')

In [ ]:
table=pd.crosstab(df.PhoneService,df.Churn)
table.div(table.sum(1).astype(float), axis=0).plot(kind='bar', stacked=True)
plt.title('Churn Frequency for PhoneService')
plt.xlabel('PhoneService')
plt.ylabel('Frequency of Churn')
plt.savefig('churn_frequency_per_PhoneService')

## Create dummy variables

In [10]:
cat_vars=['gender','Partner','Dependents','PhoneService','MultipleLines','InternetService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','Contract','PaperlessBilling','PaymentMethod','SeniorCitizen']
# criamos uma lista com as variáveis categóricas do df

for var in cat_vars:
    cat_list = pd.get_dummies(df[var], prefix=var) # método get_dummies cria variáveis binárias para cada variável categórica  do df
    df=df.join(cat_list) #concatena as colunas dummies com o df original
data_vars=df.columns.values.tolist() #armazena todas as variáveis e colunas do df
to_keep=[i for i in data_vars if i not in cat_vars] #armazena todas as variáveis e nomes das colunas que são numéricas

In [11]:
data_final=df[to_keep] # criando lista que será usada para verificar a relevancia das variaveis para o algoritmo
data_final.columns.values

array(['tenure', 'MonthlyCharges', 'TotalCharges', 'Churn',
       'gender_Female', 'gender_Male', 'Partner_No', 'Partner_Yes',
       'Dependents_No', 'Dependents_Yes', 'PhoneService_No',
       'PhoneService_Yes', 'MultipleLines_No',
       'MultipleLines_No phone service', 'MultipleLines_Yes',
       'InternetService_DSL', 'InternetService_Fiber optic',
       'InternetService_No', 'OnlineSecurity_No',
       'OnlineSecurity_No internet service', 'OnlineSecurity_Yes',
       'OnlineBackup_No', 'OnlineBackup_No internet service',
       'OnlineBackup_Yes', 'DeviceProtection_No',
       'DeviceProtection_No internet service', 'DeviceProtection_Yes',
       'TechSupport_No', 'TechSupport_No internet service',
       'TechSupport_Yes', 'StreamingTV_No',
       'StreamingTV_No internet service', 'StreamingTV_Yes',
       'StreamingMovies_No', 'StreamingMovies_No internet service',
       'StreamingMovies_Yes', 'Contract_Month-to-month',
       'Contract_One year', 'Contract_Two year', 'P

## Over-sampling using SMOTE

With our training data created, I’ll up-sample the no-churn using the SMOTE algorithm(Synthetic Minority Oversampling Technique). At a high level, SMOTE:

* Works by creating synthetic samples from the minor class (no churn) instead of creating copies.
* Randomly choosing one of the k-nearest-neighbors and using it to create a similar, but randomly tweaked, new observations.

In [12]:
X = data_final.loc[:, data_final.columns != 'Churn'] # declara as variáveis independentes x
y = data_final.loc[:, data_final.columns == 'Churn'] # declara a variável dependente y (Churn)

from imblearn.over_sampling import SMOTE # importa a classe SMOTE responsável pelo balanceamento de datasets

os = SMOTE(random_state=0) # cria uma instância da classe SMOTE com um valor aleatório
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0) # divide o dataset em treino e teste usado a divisão fixa (70% treino e 30% teste)
columns = X_train.columns # armazena os nomes das colunas das características do conjunto de treinamento (X_train)..
os_data_X,os_data_y=os.fit_resample(X_train, y_train) # aplica o SMOTE nos dados de treino para fazer o oversampling da classe minoritária balanceando o algoritmo
os_data_X = pd.DataFrame(data=os_data_X,columns=columns ) # converte as variaveis sinteticas em df usando os nomes originais das colunas armazenados na variável "columns".
os_data_y= pd.DataFrame(data=os_data_y,columns=['Churn']) # converte a variável alvo sintetica em um df

# Verificando os resultados do oversampling
print("length of oversampled data is ",len(os_data_X))
print("Number of no churn in oversampled data",len(os_data_y[os_data_y['Churn']==0]))
print("Number of churn",len(os_data_y[os_data_y['Churn']==1]))
print("Proportion of no churn data in oversampled data is ",len(os_data_y[os_data_y['Churn']==0])/len(os_data_X))
print("Proportion of churn data in oversampled data is ",len(os_data_y[os_data_y['Churn']==1])/len(os_data_X))

length of oversampled data is  7216
Number of no churn in oversampled data 3608
Number of churn 3608
Proportion of no churn data in oversampled data is  0.5
Proportion of churn data in oversampled data is  0.5


C:\Users\Maiara\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\Maiara\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


In [ ]:
columns

## Recursive Feature Elimination

In [13]:
data_final_vars=data_final.columns.values.tolist() # obtém uma lista com os nomes das colunas presentes no df data_final.
y=['Churn'] #Aqui, a variável Churn é definida como uma lista para ser prevista pelo modelo
X=[i for i in data_final_vars if i not in y] # cria a lista X que conterá todos os atributos que serão usados como entrada para o modelo

from sklearn.feature_selection import RFE # classe RFE (Recursive Feature Elimination) que visa eliminar as características menos importantes ou irrelevantes para melhorar o desempenho do modelo.
from sklearn.linear_model import LogisticRegression # classe LogisticRegression que será usada como o modelo de regressão logística para a seleção de atributos.

logreg = LogisticRegression() # Cria uma instância do modelo de Regressão Logística.
rfe = RFE(logreg, step = 15) # Cria um objeto RFE e especifica que desejamos selecionar 5 atributos menos importantes para a remoção em cada interação
rfe = rfe.fit(os_data_X, os_data_y.values.ravel()) # Aqui, o objeto RFE é ajustado aos dados. os_data_X representa as features do conjunto de treinamento após a aplicação da técnica SMOTE e os_data_y é o respectivo target após o mesmo processo.

print(rfe.support_) #Exibe um array booleano que indica quais atributos foram selecionados pelo RFE. O valor True em uma determinada posição do array significa que o atributo correspondente foi selecionado.
print(rfe.ranking_) #Exibe o ranking dos atributos. Aqueles com ranking 1 são considerados os mais importantes, e aqueles com valores mais altos são considerados menos importantes.

[False False False  True  True  True  True  True  True False  True  True
 False  True  True  True False  True False  True  True False  True  True
 False  True  True False False False False False False False False  True
  True  True False  True False False  True False False False]
[2 3 3 1 1 1 1 1 1 3 1 1 3 1 1 1 3 1 3 1 1 3 1 1 3 1 1 3 2 3 3 2 2 3 2 1 1
 1 3 1 2 2 1 3 3 2]


C:\Users\Maiara\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Maiara\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/st

In [14]:
rfe_ranking = rfe.ranking_ #armazenando as posições do rank

In [15]:
selected_features = []

for i, rank in enumerate(rfe_ranking):
    if rank == 1:  # Verificar se o valor em rfe.ranking_ é igual a 1
        selected_features.append(columns[i])  # Se sim, adicionar o valor correspondente em "column"s à lista selected_features

print(selected_features)

['gender_Female', 'gender_Male', 'Partner_No', 'Partner_Yes', 'Dependents_No', 'Dependents_Yes', 'PhoneService_Yes', 'MultipleLines_No', 'MultipleLines_Yes', 'InternetService_DSL', 'InternetService_Fiber optic', 'OnlineSecurity_No', 'OnlineSecurity_Yes', 'OnlineBackup_No', 'OnlineBackup_Yes', 'DeviceProtection_No', 'DeviceProtection_Yes', 'TechSupport_No', 'Contract_Month-to-month', 'Contract_One year', 'Contract_Two year', 'PaperlessBilling_Yes', 'PaymentMethod_Electronic check']


## Implementing the model

In [16]:
cols = selected_features # seleciona apenas as variáveis relevantes
X=os_data_X[cols] # x recebe apenas as variáveis relevantes
y=os_data_y['Churn'] # y recebe a variável alvo

In [ ]:
X

## Implementing the model

In [17]:
import statsmodels.api as sm # statsmodels é uma lib que fornece várias funções e classes para realizar análises estatísticas e ajuste de modelos.
logit_model=sm.Logit(y,X) # criando um objeto logit_model que representa o modelo de regressão logística. O primeiro argumento y é o vetor que contém a variável resposta (variável dependente) do modelo, e o segundo argumento X é a matriz (ou DataFrame) que contém as variáveis preditoras (variáveis independentes). O modelo logit é usado para modelar a relação entre a variável resposta binária e as variáveis preditoras.
result=logit_model.fit() # ajustando o modelo logit aos dados fornecidos. O método fit() realiza a estimação dos parâmetros do modelo usando os dados de entrada X e y. Ele encontra os coeficientes que melhor se ajustam aos dados para descrever a relação entre a variável resposta binária e as variáveis preditoras.
print(result.summary2()) # O método summary2() gera uma tabela com várias informações úteis sobre o modelo, incluindo os coeficientes estimados, seus erros padrão, estatísticas de teste (como o valor Z), valores p, intervalos de confiança para os coeficientes e outras métricas relevantes. 

         Current function value: 0.428583
         Iterations: 35
                                     Results: Logit
Model:                      Logit                    Method:                   MLE      
Dependent Variable:         Churn                    Pseudo R-squared:         0.382    
Date:                       2023-07-23 18:03         AIC:                      6231.3093
No. Observations:           7216                     BIC:                      6389.6426
Df Model:                   22                       Log-Likelihood:           -3092.7  
Df Residuals:               7193                     LL-Null:                  -5001.8  
Converged:                  0.0000                   LLR p-value:              0.0000   
No. Iterations:             35.0000                  Scale:                    1.0000   
----------------------------------------------------------------------------------------
                                Coef.    Std.Err.    z     P>|z|     [0.025     0

C:\Users\Maiara\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [18]:
cols = [col for col, selected in zip(cols, selected_features) if selected and col not in ["PhoneService_Yes", "MultipleLines_No","MultipleLines_Yes"]]
#retirando as variaveis com p-value > 0.5
X=os_data_X[cols]
y=os_data_y['Churn']
logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.473449
         Iterations 7
                                Results: Logit
Model:                   Logit                 Method:                MLE      
Dependent Variable:      Churn                 Pseudo R-squared:      0.317    
Date:                    2023-07-23 18:04      AIC:                   6872.8183
No. Observations:        7216                  BIC:                   7010.4995
Df Model:                19                    Log-Likelihood:        -3416.4  
Df Residuals:            7196                  LL-Null:               -5001.8  
Converged:               1.0000                LLR p-value:           0.0000   
No. Iterations:          7.0000                Scale:                 1.0000   
-------------------------------------------------------------------------------
                                Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
---------------------------------------------------

## Logistic Regression Model Fitting

In [21]:
from sklearn.linear_model import LogisticRegression # classe utilizada para criar e treinar um modelo de regressão logística
from sklearn import metrics # a classe contém várias funções para avaliar a performance do modelo.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0) # dividindo os dados em conjunto de treinamento e conjunto de teste com a função train_test_split() 
# Os argumentos X e y representam as variáveis preditoras e a variável resposta, respectivamente. O argumento test_size=0.3 
# define que 30% dos dados serão utilizados para formar o conjunto de teste, enquanto o restante (70%) 
# será utilizado para treinar o modelo. O argumento random_state=0 é usado para garantir que a divisão 
# seja feita de forma determinística, ou seja, sempre obteremos a mesma divisão de treinamento/teste quando executarmos o código novamente.

logreg = LogisticRegression() # o objeto logreg da classe LogisticRegression representa o modelo de regressão logística que será treinado. 

# A função fit() é usada para treinar o modelo com os dados de treinamento (X_train e y_train). Durante o treinamento, o modelo aprenderá os coeficientes (pesos) que melhor se ajustam aos dados de treinamento, de forma a realizar a previsão da variável resposta com base nas variáveis preditoras.
logreg.fit(X_train, y_train) #Após a execução do código, o objeto logreg conterá o modelo de regressão logística treinado, pronto para fazer previsões em novos dados não vistos.

LogisticRegression()

In [20]:
y_pred = logreg.predict(X_test) # A função predict() é aplicada ao modelo logreg com X_test como argumento, e ela retorna uma matriz contendo as previsões do modelo para o conjunto de teste. Essas previsões são armazenadas na variável y_pred.
print('Avurácia do classificador de regressão logística no conjunto de teste.: {:.2f}'.format(logreg.score(X_test, y_test))) # calculando a acurácia do modelo de regressão logística no conjunto de teste. A acurácia é usada para avaliar a performance de modelos de classificação. Ela mede a proporção de previsões corretas em relação ao total de previsões feitas pelo modelo.
#O método score() é aplicado ao modelo logreg com X_test e y_test como argumentos. Ele compara as previsões do modelo (armazenadas em y_pred) com as verdadeiras classes do conjunto de teste (y_test) e calcula a acurácia como a proporção de previsões corretas em relação ao total de amostras no conjunto de teste.

Accuracy of logistic regression classifier on test set: 0.84


## Confusion Matrix

In [23]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred) # A função confusion_matrix é usada para calcular a matriz de confusão, que é uma tabela que descreve o desempenho de um modelo de classificação em termos de verdadeiros positivos (TP), falsos positivos (FP), verdadeiros negativos (TN) e falsos negativos (FN).
print(confusion_matrix)

[[888 176]
 [160 941]]


O resultado nos mostra que temos 888+941 predições corretas e 160+176 predições incorretas.

* São 888 verdadeiros negativos
* 160 falsos negativos
* 176 falsos positivos
* 941 verdadeiros positivos

## Compute precision, recall, F-measure and support

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))


Tradução de cada métrica conforme a citação do Scikit Learn:

* Precisão (Precision):
A precisão é a razão tp / (tp + fp), onde tp é o número de verdadeiros positivos e fp é o número de falsos positivos. A precisão é intuitivamente a habilidade do classificador de não rotular uma amostra como positiva se ela for negativa.

* Recall (Revocação):
A revocação é a razão tp / (tp + fn), onde tp é o número de verdadeiros positivos e fn é o número de falsos negativos. A revocação é intuitivamente a habilidade do classificador de encontrar todas as amostras positivas.

* F-measure (F-medida):
A F-medida é interpretada como uma média harmônica ponderada da precisão e da revocação, onde a F-medida atinge seu melhor valor em 1 e pior valor em 0.

* Suporte (Support):
O suporte é o número de ocorrências de cada classe em y_test.

Em resumo, essas métricas são utilizadas para avaliar o desempenho de um classificador em tarefas de classificação. A precisão mede a proporção de verdadeiros positivos entre todas as amostras classificadas como positivas, enquanto a revocação mede a proporção de verdadeiros positivos encontrados entre todas as amostras verdadeiramente positivas. A F-medida combina a precisão e a revocação em uma única métrica, sendo útil quando se deseja equilibrar o trade-off entre ambas. O suporte informa quantas amostras existem para cada classe na coleção de teste (y_test).

Do dataset completo, 82% dos clientes testados realmente resultaram em Churn para a empresa.

## ROC Curve

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Regressão Logística (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taxa de falsos positivos')
plt.ylabel('Taxa de verdadeiros positivos')
plt.title('Característica de Operação do Receptor')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()